<a href="https://colab.research.google.com/github/shu65/theoretical-numerical-linear-algebra/blob/main/%E7%B7%9A%E5%BD%A2%E8%A8%88%E7%AE%97%E3%81%AE%E6%95%B0%E7%90%86_7_2_3_%E3%81%B9%E3%81%8D%E4%B9%97%E6%B3%95%E3%80%80%E3%82%B7%E3%83%95%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

a_matrix = np.array([
    [1., 0., 0.,],
    [0., 2., 1.,],
    [0., 1., 1.],
])
a_matrix

array([[1., 0., 0.],
       [0., 2., 1.],
       [0., 1., 1.]])

In [ ]:
np.linalg.eig(a_matrix)

(array([0.38196601, 2.61803399, 1.        ]),
 array([[ 0.        ,  0.        ,  1.        ],
        [ 0.52573111, -0.85065081,  0.        ],
        [-0.85065081, -0.52573111,  0.        ]]))

In [ ]:
def lu_decompose(a):
  n = a.shape[0]
  t = a.copy()
  for k in range(n):
    w = 1/t[k, k]
    for i in range(k + 1, n):
      t[i, k] = t[i, k]*w
      for j in range(k + 1, n):
        t[i,j] = t[i,j] - t[i, k] * t[k, j]
  l = np.identity(n)
  u = np.zeros_like(a)
  for i in range(n):
    for j in range(n):
      if i > j:
        l[i, j] = t[i, j]
      else:
        u[i, j] = t[i, j]
  return l, u

l, u = lu_decompose(a_matrix)
print("l", l)
print("u", u)

l [[1.  0.  0. ]
 [0.  1.  0. ]
 [0.  0.5 1. ]]
u [[1.  0.  0. ]
 [0.  2.  1. ]
 [0.  0.  0.5]]


In [ ]:
np.testing.assert_almost_equal(a_matrix, l @ u)

In [ ]:
def l_solve(l, x):
  n = len(x)
  v = np.zeros(n)
  a = l.copy()
  b = x.copy()
  for i in range(n):
    tmp = b[i] / a[i, i]
    v[i] = tmp
    for j in range(i+1, n):
      b[j] -= a[j, i] *  tmp
      a[j, i] = 0
  return v

x = np.ones((a_matrix.shape[1],))/np.sqrt(a_matrix.shape[1]) 
v = l_solve(l, x)
v

array([0.57735027, 0.57735027, 0.28867513])

In [ ]:
np.testing.assert_almost_equal(l @ v, x)

In [ ]:
def u_solve(u, v):
  n = len(v)
  y = np.zeros(n)
  a = u.copy()
  b = v.copy()
  for i in reversed(range(n)):
    tmp = b[i] / a[i, i]
    y[i] = tmp
    for j in range(0, i):
      b[j] -= a[j, i] *  tmp
      a[j, i] = 0
    #print(a, b, y)
  return y

x = np.ones((a_matrix.shape[1],))/np.sqrt(a_matrix.shape[1]) 
y = u_solve(u, v)
y

array([0.57735027, 0.        , 0.57735027])

In [ ]:
np.testing.assert_almost_equal(u @ y, v)

In [ ]:
def shift_inverse_power_method(a, s, max_iterations=100):
  n = a.shape[0]
  l, u = lu_decompose(a - s*np.identity(n))
  eigen_vector = np.ones((a.shape[1],))/np.sqrt(a.shape[1])
  print("init eigen_vector", eigen_vector)
  for i in range(max_iterations):
    v = l_solve(l, eigen_vector)
    y = u_solve(u, v)
    inversed_eigen_value = np.linalg.norm(y)
    eigen_vector = y/inversed_eigen_value
    print(f"iter:{i} eigen_value:{1.0/inversed_eigen_value} eigen_vector:{eigen_vector}")
  
  return s + 1.0/inversed_eigen_value, eigen_vector

shift_inverse_power_method(a_matrix, s=0.999)

init eigen_vector [0.57735027 0.57735027 0.57735027]
iter:0 eigen_value:0.0017320499415415227 eigen_vector:[ 9.9999950e-01  1.0000005e-03 -1.0010015e-06]
iter:1 eigen_value:0.0010000005000008758 eigen_vector:[ 1.00000000e+00 -2.00300802e-09  1.00200601e-06]
iter:2 eigen_value:0.0010000000000005029 eigen_vector:[ 1.00000000e+00  1.00301203e-09 -1.00601805e-09]
iter:3 eigen_value:0.0010000000000000009 eigen_vector:[ 1.00000000e+00 -1.00803010e-12  2.01205016e-12]
iter:4 eigen_value:0.0010000000000000009 eigen_vector:[ 1.00000000e+00  2.01507528e-15 -3.02512046e-15]
iter:5 eigen_value:0.0010000000000000009 eigen_vector:[ 1.00000000e+00 -3.03016873e-18  5.04827418e-18]
iter:6 eigen_value:0.0010000000000000009 eigen_vector:[ 1.00000000e+00  5.05636577e-21 -8.09159087e-21]
iter:7 eigen_value:0.0010000000000000009 eigen_vector:[ 1.00000000e+00 -8.10476010e-24  1.31692306e-23]
iter:8 eigen_value:0.0010000000000000009 eigen_vector:[ 1.00000000e+00  1.31905391e-26 -2.13084898e-26]
iter:9 eigen_v

(1.0, array([ 1.00000000e+000, -1.59085639e-280,  2.57405971e-280]))